In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error

# скроем предупреждения о возможных ошибках для лучшей читаемости
import warnings
warnings.filterwarnings('ignore')


# Ансамбли моделей машинного обучения.

Возьмем набор данных из предыдущей работы, проведем кодирование категориальных признаков и масштабирование данных и разделим данные на обучающую и тестовую выборки.

Загрузили датасет, содержащий данные НАСА о звездах. Датасет содержит следующие столбцы:
* Temperature - температура в Кельвинах
* L - относительная яркость
* R - относительный радиус
* A_M - абсолютная звездная величина
* Color - цвет
* Spectral_Class - спектральный класс
* Type - тип

In [2]:
stars = pd.read_csv("star-type-classification/Stars.csv")
stars.head()
def accuracy_score_for_classes(y_true: np.ndarray,  y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    df = y_true.copy()
    df.rename(columns={'Type':'t'}, inplace=True)
    df.insert(loc=0, column='p', value=y_pred)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res


def print_metric(y_true: np.ndarray, y_pred: np.ndarray):
    print (accuracy_score(y_true, y_pred))
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [3]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
oe = OrdinalEncoder()
stars[["Color", "Spectral_Class"]] = oe.fit_transform(stars[["Color", "Spectral_Class"]])
mms = MinMaxScaler()
stars[['Temperature','L','R','A_M','Color','Spectral_Class']] = mms.fit_transform(
    stars[['Temperature','L','R','A_M','Color','Spectral_Class']])
s_feat = stars[["Temperature", "L", "R", "Color", "Spectral_Class", "A_M"]]
s_targ = stars[["Type"]]

x_train, x_test, y_train, y_test = train_test_split(s_feat, s_targ, random_state=1)


## Обучение моделей

Бэггинг для задачи классификации

In [4]:
bc1 = BaggingClassifier(n_estimators=5, oob_score=True, random_state=10)
bc1.fit(x_train, y_train)
#bc1.predict(x_test)
print_metric(y_test, bc1.predict(x_test))

In [5]:
print_metric(y_test, bc1.estimators_[0].predict(x_test))

In [6]:
print_metric(y_test, bc1.estimators_[1].predict(x_test))

In [7]:
import graphviz
import pydotplus
from IPython.display import Image
from io import StringIO 
from sklearn.tree import export_graphviz
feature_names = list(x_train)
target_names = list(y_train)


# Визуализация дерева
def get_png_tree(tree_model_param, feature_names_param):
    dot_data = StringIO()
    export_graphviz(tree_model_param, out_file=dot_data, feature_names=feature_names_param,
                    filled=True, rounded=True, special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    return graph.create_png()



def show_tree(tree, features, target):
    dot_data = export_graphviz(tree, out_file=None, 
                           feature_names=features, 
                           class_names=target, 
                           filled=True, rounded=True, special_characters=True)  
    graph = graphviz.Source(dot_data)  
    graph

In [8]:
Image(get_png_tree(bc1.estimators_[0], feature_names), width='80%')

In [9]:
Image(get_png_tree(bc1.estimators_[1], feature_names), width='80%')

Видно, что ансамбль состоит из различных моделей.

Градиентный бустинг:

In [10]:
gbc = GradientBoostingClassifier(n_estimators=5)
gbc.fit(x_train, y_train)
gbc.score(x_test, y_test)

In [11]:
print_metric(y_test, gbc.predict(x_test))

Эта модель менее точна, чем предыдущая.

Стекинг

In [12]:
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
from heamy.dataset import Dataset

In [13]:
dataset = Dataset(x_train, y_train, x_test)
# модели первого уровня
model_tree = Regressor(dataset=dataset, estimator=DecisionTreeRegressor, name='tree')
model_lr = Regressor(dataset=dataset, estimator=LinearRegression,name='lr')
model_rf = Regressor(dataset=dataset, estimator=RandomForestRegressor, parameters={'n_estimators': 5},name='rf')

In [ ]:
#Первый уровень - две модели: дерево и линейная регрессия
# Второй уровень: линейная регрессия

pipeline = ModelsPipeline(model_tree, model_lr)
stack_ds = pipeline.stack(k=10, seed=1)
# модель второго уровня
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression)
results = stacker.validate(k=10,scorer=mean_absolute_error)


In [ ]:
# Первый уровень - две модели: дерево и линейная регрессия
# Второй уровень: случайный лес

stacker = Regressor(dataset=stack_ds, estimator=RandomForestRegressor)
results = stacker.validate(k=10,scorer=mean_absolute_error)

In [ ]:
 #Первый уровень - три модели: дерево, линейная регрессия и случайный лес
# Второй уровень: линейная регрессия
pipeline = ModelsPipeline(model_tree, model_lr, model_rf)
stack_ds3 = pipeline.stack(k=10, seed=1)
# модель второго уровня
stacker = Regressor(dataset=stack_ds3, estimator=LinearRegression)
results = stacker.validate(k=10,scorer=mean_absolute_error)

Наилучший результат получен при использовании дерева и линейной регрессии на первом уровне и случайного леса на втором.